# Sentiment Analysis: Large Movie Review Dataset

Hi again! You will be expected to finish this on your own, but you can use the available channels on Discord to ask questions and help others. Please read the entire notebook before starting, this will give you a better idea of what you need to accomplish.

This project is related to NLP. As you may already know, the most important and hardest part of an NLP project is pre-processing, which is why we are going to focus on that.

Regarding the data, we are not going to have a __csv file__, that would be too easy :) instead we are going to download the data from [AI Stanford Dataset](https://ai.stanford.edu/~amaas/data/sentiment/). When you download them you will notice that their format is text files, so you will have to work a little there to be able to use and process them. This is a dataset for __binary sentiment classification__.

Basically a basic sentiment analysis problem, as in this case, consists of a classification problem, where the possible output labels are: `positive` and `negative`. Which indicates, if the review of a movie speaks positively or negatively. In our case it is a binary problem, but one could have many more "feelings" tagged and thus allow a more granular analysis.

### These are the objectives of the project:

* Read data that is not in a traditional format.
* Put together a set of preprocessing functions that we can use later on any NLP or related problems.
* Vectorize the data in order to apply a machine learning model to it: using BoW or TF-IDF.
* BoW and TF-IDF are classic ways to vectorize text, but currently we have some more complex ways with better performance, for this we are going to train our own word embedding and use it as a vectorization source for our data.
* Train a sentiment analysis model that allows us to detect positive and negative opinions in movie reviews.

In [ ]:
import urllib.request
response = urllib.request.urlopen('https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')
html = response.read()

In [ ]:
import urllib.request
import os

folder_name = 'data'

folder_path = os.path.join(folder_name)

if not os.path.exists(folder_path):
    os.mkdir(folder_path)

urllib.request.urlretrieve("https://raw.githubusercontent.com/dnishimoto/python-deep-learning/master/list%20iterators%20and%20generators.ipynb", f"{folder_path}/test.ipynb")

---
## 1. Get the data

#### Download the data and process it in order to obtain:

* `X_train:` list with reviews for training.
* `y_train:` list with labels for training.
* `X_test:` list with reviews for testing.
* `y_test:` list with labels for testing.

`Notes:` Use the target column as `positive`, that way the positive value will be indicated with a value of `1` and negative with a value of `0`. In this case, a split train/test is not necessary because the original data is already separated.

In [ ]:
import os
from os import listdir
from os import walk
import pandas as pd
from os.path import isfile, join
import glob
import pathlib
import numpy as np
from numpy import hstack

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
def get_list_filenames(directory: str, separator: str) -> list:
    """
    This function will return all the filenames of the specified directory.

    It uses os.walk to walkthrough over the entire directory and get only the files that end with .txt.
        
    Return
    ------------------------------
    list_of_filenames: list with filenames
    """
    list_of_filenames = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                list_of_filenames.append(os.path.join(root, file).split(separator)[-1])

    return list_of_filenames


* Get filenames of positive & negative reviews from training directory:

In [ ]:
pos_directory = "C:/Users/Fede/Desktop/FOCUS/AnyoneAI/SprintV/Project_006_Sentiment_Analysis/aclImdb/train/pos"
separator = '\\'
list_of_pos_filenames = get_list_filenames(directory=pos_directory, separator=separator)
print(f"Total count of filenames in train dataset of positives reviews: {len(list_of_pos_filenames)}")

In [ ]:
neg_directory = "C:/Users/Fede/Desktop/FOCUS/AnyoneAI/SprintV/Project_006_Sentiment_Analysis/aclImdb/train/neg"
separator = '\\'
list_of_neg_filenames = get_list_filenames(directory=neg_directory, separator=separator)
print(f"Total count of filenames in train dataset of negative reviews: {len(list_of_neg_filenames)}")

* Get filenames of positive & negative reviews from test directory:

In [ ]:
pos_directory_test = "C:/Users/Fede/Desktop/FOCUS/AnyoneAI/SprintV/Project_006_Sentiment_Analysis/aclImdb/test/pos"
separator = '\\'
list_of_pos_filenames_test = get_list_filenames(directory=pos_directory_test, separator=separator)
print(f"Total count of filenames in test dataset of positives reviews: {len(list_of_pos_filenames)}")

In [ ]:
neg_directory_test = "C:/Users/Fede/Desktop/FOCUS/AnyoneAI/SprintV/Project_006_Sentiment_Analysis/aclImdb/test/neg"
separator = '\\'
list_of_neg_filenames_test = get_list_filenames(directory=neg_directory_test, separator=separator)
print(f"Total count of filenames in test dataset of negative reviews: {len(list_of_neg_filenames)}")

In [ ]:
def get_list_of_corpus(list_of_filenames: list, directory: str) -> list:
    """
    Get the content of each file (i.e: the entire review of a moovie).
    Each content will be appended to a list as a unique element, despite being strings or integers.
        
    Return
    ------------------------------
    list_of_corpus: list 
    """
    list_of_corpus = []
    for file in list_of_filenames:
        try:
            with open(f"{directory}/{file}", encoding="utf8") as f:
                list_of_corpus.append(f.read())
            print(f"File {file} succesfully appended")
        except:
            print(f'The file {file} is not in the specified directory')

    return list_of_corpus

* Get corpus of each file (into a list) from training filename lists:

In [ ]:
list_of_positive_corpus = get_list_of_corpus(list_of_filenames=list_of_pos_filenames, directory=pos_directory)

In [ ]:
list_of_negative_corpus = get_list_of_corpus(list_of_filenames=list_of_neg_filenames, directory=neg_directory)

* Get corpus of each file (into a list) from test training lists:

In [ ]:
list_of_positive_corpus_test = get_list_of_corpus(list_of_filenames=list_of_pos_filenames_test, directory=pos_directory_test)

In [ ]:
list_of_negative_corpus_test = get_list_of_corpus(list_of_filenames=list_of_neg_filenames_test, directory=neg_directory_test)

* Bring together negative and positive train reviews into one list:

In [ ]:
for review in list_of_negative_corpus:
    list_of_positive_corpus.append(review)

list_of_corpus_final = list_of_positive_corpus

* Bring together negative and positive test reviews into one list:

In [ ]:
for review in list_of_negative_corpus_test:
    list_of_positive_corpus_test.append(review)

list_of_corpus_final_test = list_of_positive_corpus_test

* Create a positive & negative array (full of 1 and 0 values respectively):

In [ ]:
positive_array = np.full(shape=12500,fill_value=1, dtype=np.int64)
negative_array = np.full(shape=12500,fill_value=0, dtype=np.int64)

* Sum up X_train/y_train and X_test/y_test:

In [ ]:
X_train = pd.Series(list_of_corpus_final)
y_train =  hstack([positive_array,negative_array])

In [ ]:
X_test = pd.Series(list_of_corpus_final_test)
y_test = y_train.copy()

### *Don't change anything in this block, just make it run correctly*

We are going to check that you have done it right and for that we are going to see if the dimensions match.

In [ ]:
def check_data(X_train, y_train, X_test, y_test):
    if len(X_train) == len(y_train) == len(X_test) == len(y_test) == 25000:
        print('Reading Data Success!')
    else:
        raise ValueError('Dimensions do not match!')

check_data(X_train, y_train, X_test, y_test)

---
## 2. Normalize the data

#### Create the following functions but not here in the notebook, do it in the python script called `text_normalizer.py` and import them into the notebook (this way you can build your own NLP preprocessing library). In fact, the structure of the functions is already written, you must complete them with the code that you consider necessary.

#### Respect names and minimal interfaces:

* `remove_html_tags(text):` to remove all HTML tags that may be present in text.
* `remove_accented_chars(text):` to remove accented characters from text
* `expand_contractions(text):` to expand contractions of the type, "don't" to "do not". The contractions are already defined in the "contractions.py" file.
* `lemmatize_text(text):` to lemmatize text.
* `stem_text(text):` to apply stemming (NLTK's PorterStemmer) on text.
* `remove_special_chars(text):` to remove special characters from text.
* `remove_special_chars(text, remove_digits=True):` to remove numbers, note that it is the same function to remove special characters with the addition of an argument that enables or disables the removal of numbers.
* `remove_stopwords(text, stopwords=stop_words):` to remove stopwords from text.
* `remove_extra_new_lines(text):` to remove extra newlines from text.
* `remove_extra_whitespace(text):` to remove extra whitespaces from text.

If you want to add more features that would be great, for example you could start by removing emojis, using different stemming algorithms, etc. The more functions you have the better, remember that the texts are very varied and the preprocessing depends a lot on the source of our data.

To apply each of the functions you created and pre-process the dataset, you must use the `normalize_corpus` function of the `text_normalizer.py` script. In this method each of the functions you wrote is called, in fact you must enable or disable what you consider necessary (`at this point we leave it to your free choice, for example: you can lemmatize or apply stemming or directly not apply any of the two and so on with the rest, but that is your choice`), this function simply groups the previous ones for a more simplified use.

### *Don't change anything in this block, just make it run correctly*

We are going to check that the pre-processing does what we need it to do, for this we are going to test the functions with predefined inputs and we are going to see if the outputs match what we are looking for.

Note that the functions are not defined in the notebook itself, but rather that they are in a python file and you must import them in order to use them. Same thing with names, you're going to have to name your functions the way they were named.

In [ ]:
import nltk
import text_normalizer

# Inputs
doc_html = """
<br /><br />But with plague out there and the news being kept a secret,
the New Orleans PD starts a dragnet of the city's underworld.
"""
doc_accented = "Héllo, thís is an accented sénténce."
doc_contractions = "I can't, because it doesn't work."
doc_lemma = "The striped bats are hanging on their feet for best"
doc_stem = """
Where did he learn to dance like that?
His eyes were dancing with humor.
She shook her head and danced away.
"""
doc_specials = "hello? there A-Z-R_T(,**), world, welcome to python. this **should? the next line#followed- by@ an#other %million^ %%like $this."
doc_digits = "abc123def456ghi789zero0 hello my friend number 10"
doc_stop = "He is a very good person"
doc_new_lines = """we
use
a
lot
of
lines"""
doc_spaces = "Hello           my      dear          friend"

# Outputs
good_html = """
But with plague out there and the news being kept a secret,
the New Orleans PD starts a dragnet of the city's underworld.
"""
good_accented = "Hello, this is an accented sentence."
good_contractions = "I cannot, because it does not work."
good_lemma = "the stripe bat be hang on their foot for good"
good_stem = "where did he learn to danc like that ? hi eye were danc with humor. she shook her head and danc away ."
good_specials = "hello there AZRT world welcome to python this should the next linefollowed by another million like this"
good_digits = "abcdefghizero hello my friend number "
good_stop = "good person"
good_new_lines = "we use a lot of lines"
good_spaces = "Hello my dear friend"
stop_words = nltk.corpus.stopwords.words('english')

def check_normalization():
    if good_html == text_normalizer.remove_html_tags(doc_html):
        print('[1/10] Remove HTML Success!')
    else:
        raise ValueError('[1/10] Remove HTML Fail!')
        
    if good_accented == text_normalizer.remove_accented_chars(doc_accented):
        print('[2/10] Remove Accented Success!')
    else:
        raise ValueError('[2/10] Remove Accented Fail!')
        
    if good_contractions == text_normalizer.expand_contractions(doc_contractions):
        print('[3/10] Expand Contractions Success!')
    else:
        raise ValueError('[3/10] Expand Contractions Fail!')
        
    if good_lemma == text_normalizer.lemmatize_text(doc_lemma):
        print('[4/10] Lemmatization Success!')
    else:
        raise ValueError('[4/10] Lemmatization Fail!')
        
    print(text_normalizer.stem_text(doc_stem))
    if good_stem == text_normalizer.stem_text(doc_stem):
        print('[5/10] Stemming Success!')
    else:
        raise ValueError('[5/10] Stemming Fail!')
        
    if good_specials == text_normalizer.remove_special_chars(doc_specials):
        print('[6/10] Remove Specials Success!')
    else:
        raise ValueError('[6/8] Remove Specials Fail!')
        
    if good_digits == text_normalizer.remove_special_chars(doc_digits, remove_digits=True):
        print('[7/10] Remove Digits Success!')
    else:
        raise ValueError('[7/10] Remove Digits Fail!')
        
    if good_stop == text_normalizer.remove_stopwords(doc_stop, stopwords=stop_words):
        print('[8/10] Remove Stopwords Success!')
    else:
        raise ValueError('[8/10] Remove Stopwords Fail!')
    
    if good_new_lines == text_normalizer.remove_extra_new_lines(doc_new_lines):
        print('[9/10] Remove New Lines Success!')
    else:
        raise ValueError('[9/10] Remove New Lines Fail!')
        
    if good_spaces == text_normalizer.remove_extra_whitespace(doc_spaces):
        print('[10/10] Remove Extra Whitespaces Success!')
    else:
        raise ValueError('[10/10] Remove Extra Whitespaces Fail!')

check_normalization()

---
## 3. Feature Engineering

You already have the pr-eprocessed data, now you must vectorize them, because remember that the models only understand numbers. At this stage choose whether you want to vectorize with BoW or with TF-IDF. Later we will train our own embedding but for now we go with a more "classic" vectorization.

In [ ]:
from text_normalizer import normalize_review
from concurrent.futures import ProcessPoolExecutor
from sklearn.feature_extraction.text import TfidfVectorizer

* Parallelize normalize texts to **enhance pre-processing time**: (both X_train/X_test)

In [ ]:
with ProcessPoolExecutor(max_workers=10) as executor:
    X_train_preprocessed = list(executor.map(normalize_review, X_train))

In [ ]:
with ProcessPoolExecutor(max_workers=10) as executor:
    X_test_preprocessed = list(executor.map(normalize_review, X_test))

* Instantiate vectorizer TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(norm = None)

* Fit & transform X_train from X_train_preprocessed (which is X_train previously preprocessed)

In [ ]:
X_train_proc_vect = vectorizer.fit_transform(X_train_preprocessed)

* Same with X_test

In [ ]:
X_test_proc_vect = vectorizer.transform(X_test_preprocessed)

## 4. Modeling and Performance Evaluation

As we said at the beginning, what interests us most in this part is pre-processing. However, we must train a model, so choose a model of your choice (obviously a classification model, given the problem we are facing) and apply everything we learned. Also if you want you can try several models, the more models you use and know better!

In addition to training the model we ask you to show:

* `Precision`
* `Recall`
* `F1-Score`
* `Classification Report`
* `Confusion Matrix`

To do this you must complete the `get_performance` function of the `evaluation.py` script.

Also, you must complete the `plot_roc` function so that it can show:

* `ROC Curve`
* `Obtain the ROC-AUC value (later we will do a small minimum performance check with this value)`

In [ ]:
from sklearn.metrics import roc_auc_score
from evaluation import get_performance, plot_roc
import lightgbm as lgbm

* In this case, we are going to use LightGBM as our main classifier. The main reason is that it's easy to enable the GPU as the computing device

In [ ]:
model = lgbm.LGBMClassifier(device="gpu" ,learning_rate= 0.05, max_depth= -1, num_iterations=150, num_leaves= 100, verbose=1)

* We fit our function from X_train_proc_vect to y_train. Then we calculate the probability of our predictions, slicing the "second" array to grab only those values which represents the target label.

In [ ]:
model.fit(X_train_proc_vect,y_train)

lgbm_probs = model.predict_proba(X_test_proc_vect)[:,1]

* We bring up a performance measurement to observe our model's accuracy. In this case, we are going to use **ROC AUC** Score. In addition, we call get_performance, a function that serves us a report of our algorithm 

In [ ]:
roc_auc = roc_auc_score(y_test,lgbm_probs)
print(f"AUC ROC Score: {round(roc_auc,4)}")
get_performance(predictions=lgbm_probs.round(),y_test=y_test)

* Let's plot our ROC Curve

In [ ]:
roc_auc = plot_roc(model=model, y_test=y_test, features=X_test_proc_vect)

### *Don't change anything in this block, just make it run correctly*

Let's check that the `get_performance` function returns the metrics correctly. For that we are going to simulate input/output data of a model.

In [ ]:
import evaluation

In [ ]:
def check_basic_metrics():
    accuracy, precision, recall, f1_score = evaluation.get_performance(
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], 
        [1, 1, 1, 0, 0, 1, 1, 0, 0, 0]
    )
    
    if (accuracy, precision, recall, f1_score) == (0.6, 0.6, 0.6, 0.6):
        print('Success!')
    else:
        raise ValueError('You must check your get_performance function!')
        
check_basic_metrics()

Let's see if your model at least exceeds an ROC-AUC of 0.93

In [ ]:
def check_roc(roc_auc):
    if roc_auc > 0.93:
        print('Success!')
    else:
        raise ValueError('Your model is not good enough!')
        
check_roc(roc_auc)

---
## 5. Feature Engineering with Custom Word Embedding

### Tokenize reviews and train your own Word Embedding

You are going to have to train your own word embedding, for this we are going to use the __gensim__ library. The only requirement we ask of you is that the $vector\_size=100$.

[Here](https://radimrehurek.com/gensim/models/word2vec.html) you can read Gensim's Word2Vec documentation so you can train your own embedding, using the review data as a corpus.

As a previous step to training your word embedding you must tokenize the corpus, this may take a bit depending on the size of the dataset and the tokenizer we use, if you want you can try the NLTK tokenizer called `ToktokTokenizer`, which turns out to be a little faster (we hope that this recommendation does not bias your work, try and use the ones you want)

In [ ]:
from gensim.models import Word2Vec
from nltk import word_tokenize
from gensim.utils import effective_n_jobs
from sklearn.metrics import confusion_matrix
import seaborn as sns
print(f'Number of cores availables: {effective_n_jobs(-1)}')
import numpy

* Before star training our Word2Vec vectorization, we have to tokenize X_train & X_test (preciously preprocessed)

In [ ]:
X_train_tokenized = [word_tokenize(token) for token in X_train_preprocessed]

In [ ]:
X_test_tokenized = [word_tokenize(token) for token in X_test_preprocessed]

* Instantiate our Word2Vec model. In this case I defined the parameter **sg** as 1, this means that the model will understand each word to predict another context words. Skipgram is better to capture semantic relationships meanwhile studies the context from which the words come. For the sake of the problem we are facing (sentiment analysis), I assume this algorithm will perform better than CBoW (which does the opposite and therefore, will perform better to predict syntactic words in a given context)

In [ ]:
model_w2v = Word2Vec(sentences= X_train_tokenized, sg=1, workers=12, vector_size=100)

### Generate averaged word vector features

Once the embedding has been trained, we must use it. Remember that embedding will convert each word you pass to it into a vector of a given dimension (in our case $vector\_size=100$). So in order to obtain a vector for each review, you must average the vectors of all the words that are part of the same review.

The function must have the following form:
* `vectorizer(corpus, model, num_features=100)`


Where:
* `corpus:` corresponds to the entire dataset, in this way we obtain an average vector for each review, with a single call to the function.
* `model:` is your trained model.
* `num_features:` the dimension of the output vector of your embedding (remember that in our case we set this value to 100).

In [ ]:
def vectorizer(corpus, model, num_features=100):
    
    corpus_vectors = []
    for token in corpus:
        test = np.average([model.wv[word] for word in token if word in model.wv], axis=0)
        corpus_vectors.append(test)
        
    return corpus_vectors

In [ ]:
def vectorizer(corpus, model, num_features=100):
    """
    Get the embedding of each sentence. 
    
    It iterates over the sentence and calculate the vector for each word, then, sum that word vector to the final vector composed of 100 dimensions and calculate the average from all vectors.

        
    Return
    ------------------------------
    corpus_vectors: list of arrays of 100 dim each one
    """
    
    vector_sum = np.zeros(shape=(num_features,))
    corpus_vectors = list()

    # Loop to average the vectors in a sentence
    for sentence in corpus:
        counter = 0
        for token in sentence:
            if token in model.wv:
                word_vector = model.wv[token]          
                vector_sum = vector_sum + word_vector   
                counter += 1
        average = vector_sum/counter                
        corpus_vectors.append(average) 

    return corpus_vectors

### *Don't change anything in this block, just make it run correctly*

Let's do a simple check of the embedding that you just trained.

In [ ]:
def check_embedding(model):
    vector = model.wv['computer']
    if len(vector) == 100:
        print(
            'Success! Your embedding tells me that "women" and "man" '
            f'are similar with a score of {model.wv.similarity("woman", "man")}'
        )
    else:
        raise ValueError('You should check your embedding vector size!')
        
check_embedding(model_w2v)

Now, we are going to check the vectorizer, remember that the vectorizer must generate an average vector of all the words present in the same review. So we're going to get two vectors of two words and manually average them, then using those two words we'll simulate a tokenized sentence and see that it matches the manual way.

In [ ]:
def check_vectorizer(model):
    vector1 = model.wv['personal']
    vector2 = model.wv['computer']
    avg = vectorizer([['personal', 'computer']], model)[0]

    if np.allclose((vector1 + vector2) / 2, avg):
        print('Success!')
    else:
        raise ValueError('You should check your vectorizer!')
        
check_vectorizer(model_w2v)

### Modeling

Finally train a new model, it can be the same one you used before and compare the results you got using BoW/TF-IDF and Word2Vec.

In addition to training the model we ask you to show:

* `Accuracy`
* `Recall`
* `F1-Score`
* `Classification Report`
* `Confusion Matrix`
* `ROC Curve`
* `Obtain the ROC-AUC value (later we will do a small minimum performance check with this value)`

In [ ]:
model = lgbm.LGBMClassifier(device="gpu" ,learning_rate= 0.05, max_depth= -1, num_iterations=150, num_leaves= 100, verbose=1)

* We calculate the average embedding for each sentence from the tokenized X_train and X_test

In [ ]:
X_train_word2vec = vectorizer(corpus=X_train_tokenized, model=model_w2v, num_features=100)

In [ ]:
X_test_word2vec = vectorizer(corpus=X_test_tokenized, model=model_w2v, num_features=100)

* Fit the function with our model and predict probabilities to target value

In [ ]:
model.fit(X_train_word2vec,y_train)

lgbm_probs = model.predict_proba(X_test_word2vec)[:,1]

* Calculate the ROC AUC Score (the higher, the better performance of the algorithm)

In [ ]:
roc_auc = roc_auc_score(y_test,lgbm_probs)
print(f"AUC ROC Score: {round(roc_auc,4)}")
get_performance(predictions=lgbm_probs.round(),y_test=y_test)

* Plot CMatrix to observe TP vs TN

In [ ]:
cf_matrix = confusion_matrix(y_test, lgbm_probs.round())

In [ ]:
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

* Plot ROC Curve

In [ ]:
roc_auc = plot_roc(model=model, y_test=y_test, features=X_test_word2vec)

### *Don't change anything in this block, just make it run correctly*

In [ ]:
check_roc(roc_auc)

---
### OPTIONAL:

In our case, we train a word embedding from scratch, which is very good at an educational level, but when applying it to a real problem, we need a lot of data (which is not the case with our problem). Therefore, we invite you to investigate and use one of the `pre-trained Word2Vec models`.

If you look for the `Pretrained models` section in this [link](https://radimrehurek.com/gensim/models/word2vec.html), you will find information about the models that Gensim owns.